# Build an efficient unsupervised word translator

Based on : "Word Translation Without Parallel Data" by Alexis Conneau, Guillaume Lample, Marc Aurelio Ranzato, Ludovic Denoyer & Hervé Jégou (2017)

In [1]:
import io

import numpy as np
import matplotlib.pyplot as plt
import math

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
#from torch.autograd.variable import Variable

from scipy.stats import special_ortho_group

#from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# load function for pretrained versions of word embeddings
def load_embeddings(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [3]:
##########
eng_path = '/Users/louismonier/Downloads/Monolingual/wiki.en.vec' 
fr_path = '/Users/louismonier/Downloads/Monolingual/wiki.fr.vec'
##########
#eng_path = '/Users/vince/DataProjetNLP/wiki.en.vec'
#fr_path = '/Users/vince/DataProjetNLP/wiki.fr.vec'
##########
nmax = 50000  # maximum number of word embeddings to load

# load monolingual word embeddings 
src_embeddings, src_id2word, src_word2id = load_embeddings(fr_path, nmax) # source = french 
tgt_embeddings, tgt_id2word, tgt_word2id = load_embeddings(eng_path, nmax) # target = english

In [4]:
# load ground-truth bilingual dictionaries function
def load_dic(path):
    dico_full = {}
    vectors_src=[]
    vectors_tgt = []
    with io.open(path,'r',encoding='utf_8') as f:
        for i,line in enumerate(f):
            word_src, word_tgt = line.rstrip().split(' ',1)
            if word_tgt in tgt_word2id :
                dico_full[word_src]=word_tgt
    for key in dico_full.keys() :
            vectors_src.append(src_embeddings[src_word2id[key]])
            vectors_tgt.append(tgt_embeddings[tgt_word2id[dico_full[key]]])
    X = np.vstack(vectors_src)
    Z = np.vstack (vectors_tgt)
    return dico_full,X,Z

In [5]:
# train & test bilingual dictionaries
##########
path_train = r'/Users/louismonier/Downloads/Monolingual/fr-en.0-5000.txt' 
path_test = r'/Users/louismonier/Downloads/Monolingual/fr-en.5000-6500.txt'
##########
#path_train = '/Users/vince/DataProjetNLP/fr-en.0-5000.txt' 
#path_test = '/Users/vince/DataProjetNLP/fr-en.5000-6500.txt'
##########

dico_train, X_train, Z_train = load_dic(path_train)
dico_test, X_test, Z_test = load_dic(path_test)

# convert embeddings vectors into torch tensors 
print(type(X_train[0]))
X_train, Z_train, X_test, Z_test = map(torch.tensor, (X_train, Z_train, X_test, Z_test)) 
print(type(X_train[0]))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [6]:
print(X_train.shape[0], "training samples")
print(X_test.shape[0], "test samples")
dim = X_train.shape[1]
print("Vectors dimension :", dim)

4971 training samples
1483 test samples
Vectors dimension : 300


## Build the discriminator 

Recall what is the objective of the discriminator here : ...

In [15]:
class Discriminator(nn.Module):
    def __init__(self, dim):
        super(Discriminator,self).__init__()
        self.h1 = nn.Linear(dim, 2048,bias=True) # 1st hidden layer
        self.h2 = nn.Linear(2048,2048,bias=True) # 2nd hidden layer
        self.out = nn.Linear(2048,1,bias=True) # output layer
        
    def forward(self, x):
        x = F.dropout(x, p = 0.1) # dropout pour ajouter du bruit
        x = F.leaky_relu(self.h1(x), negative_slope=0.2)
        x = F.leaky_relu(self.h2(x), negative_slope=0.2)
        y = torch.sigmoid(self.out(x)) # ouput = proba
        return y

In [8]:
#def LossDisc(y, y_pred):
#    return(-math.log((y_pred**y)*(1-y_pred)**(1-y)))

# or : 
LossD = nn.BCELoss()

## Build the generator 

Recall what is the objective of the discriminator here : ...

In [9]:
# simple linear function 
# can be seen at a neural network whose weights are elements of W 
class Generator(nn.Module):
    def __init__(self, dim):
        super(Generator, self).__init__()
        self.l1 = nn.Linear(dim, dim)

    def forward(self,x):
        y = self.l1(x)
        return y

In [10]:
# we could put this inside the class...

# to ensure that the matrix stays close to the manifold of orthogonal matrices after each update
def ortho_update(W, beta):
    W = (1+beta)*W - beta*torch.mm(torch.mm(W, W.t()), W)

In [11]:
#def LossGen(y, y_pred):
#    return(-math.log((y_pred**(1-y))*(1-y_pred)**y))

# or :
LossG = nn.BCELoss()

## It's time to train 

In [12]:
discrim = Discriminator(dim)
gen = Generator(dim)

In [13]:
optimD = optim.SGD(discrim.parameters(), lr=0.1)
optimG = optim.SGD(gen.parameters(), lr=0.1)

In [16]:
#for epoch in range(3): #3 Epochs 

N = 32 # nb of "fake" tgt // Wx
M = 32 # nb of "true" tgt // y

W = special_ortho_group.rvs(dim) # initialisation of W in SO(300)
W = torch.tensor(W,dtype=torch.float) # conversion to tensor
beta = 0.01

# ini the weights of the discrim
for p in discrim.parameters():
    p.data = torch.zeros(p.data.shape)

niter = 10
for iteration in range(niter):
    #if iteration % 10 == 0 :
        #print("epoch = %d, iteration = %d"%(epoch,iteration))
    
    # DISCRIMINATOR TRAINING
    for i in range(3):
        # set the discrimintator training mode to True
        discrim.train()
        
        # set descrim gradient to zero before computation at every step
        optimD.zero_grad()
        
        ## create a new batch of N fake & M true data to train the discriminator
        # generate 32 random words from the source 
        rand_src_word_id = torch.Tensor(N).random_(nmax).long()
        src_word_emb = src_embeddings[rand_src_word_id.numpy()]
        src_word_emb = torch.tensor(src_word_emb, dtype=torch.float) # conversion to tensor
    
        wsrc_gen = gen(src_word_emb) # translated words (Wx)
    
        # generate 32 random words from the target
        rand_tgt_word_id = torch.Tensor(M).random_(nmax).long()
        tgt_word_emb = tgt_embeddings[rand_tgt_word_id.numpy()]
        tgt_word_emb = torch.tensor(tgt_word_emb, dtype=torch.float) # conversion to tensor

        # concatenation of Wx and y aka traductions and tgt words
        input_tensor = torch.cat([wsrc_gen, tgt_word_emb],0)

        # output tensor is the answer the discriminator should give
        output_tensor = torch.Tensor(64).zero_().float()
        # we can smooth the answer by creating thresholds (# 0.8 # smoothing 80% # smoothing 20%)
        output_tensor[:N] = 1 # discrim should predict 100% proba of belonging to the src 
        output_tensor[N:] = 0 # discrim should predict 100% proba of belonging to the tgt 
        
        # prediction of the discriminator
        prediction = discrim(input_tensor)
        
        # compute loss & propogate backward
        loss_discrim = LossD(prediction, output_tensor)
        # loss_discrim = LossDisc(output_tensor, prediction)
        
        loss_discrim.backward()
        optimD.step()

    # GENERATOR TRAINING
    
    # set the discrimintator training mode to False 
    discrim.eval()
    
    # set descrim gradient to zero before computation at every step
    optimG.zero_grad()
    
    ## create a new batch of N fake & M true data to train the generator
    # generate 32 random words from the source 
    rand_src_word_id = torch.Tensor(N).random_(nmax).long()
    src_word_emb = src_embeddings[rand_src_word_id.numpy()]
    src_word_emb = torch.tensor(src_word_emb, dtype=torch.float) # conversion to tensor

    wsrc_gen = gen(src_word_emb) # translated words (Wx)

    # generate 32 random words from the target
    rand_tgt_word_id = torch.Tensor(M).random_(nmax).long()
    tgt_word_emb = tgt_embeddings[rand_tgt_word_id.numpy()]
    tgt_word_emb = torch.tensor(tgt_word_emb, dtype=torch.float) # conversion to tensor    
    
    # concatenation of Wx and y aka traductions and tgt words
    input_tensor = torch.cat([wsrc_gen, tgt_word_emb],0)
    
    # output tensor is the answer the discriminator should give
    output_tensor = torch.Tensor(64).zero_().float()
    # we can smooth the answer by creating thresholds (# 0.8 # smoothing 80% # smoothing 20%)
    output_tensor[:N] = 1 # discrim should predict 100% proba of belonging to the src 
    output_tensor[N:] = 0 # discrim should predict 100% proba of belonging to the tgt 
      
    # prediction of the discriminator on the new batch
    prediction = discrim(input_tensor)

    # compute loss & propogate backward
    # the discriminator is fooled if he predicts the contrary of what he should have predicted
    loss_gen = LossG(prediction, 1-output_tensor)
    # loss_gen = LossGen(1-output_tensor, prediction)

    loss_gen.backward()
    optimG.step()    

    W_trained = gen.l1.weight.data # get the weights of the generator which are the elements of W
    W_ortho = ortho_update(W_trained, beta)